In [2]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
import upsetplot

p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *

In [3]:
def clean_figure(ax):
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params(axis="x", rotation=45)

In [5]:
config = load_config()
od = '../snakemake/compare_external/'

## Get novelty compositions of the other things

In [6]:
f_dir = '/Users/fairliereese/Documents/programming/mele_lab/projects/240903_pt/ref/compare_external/'
files = [f'{f_dir}gtex_sqanti_classification.txt',
     f'{f_dir}enc_sqanti_classification.txt']
refs = ['GTEx', 'ENCODE4']
for f, r in zip(files, refs):
    df = pd.read_csv(f, sep='\t')
    print(len(df))

387944
387944
